In [1]:
import os
from neo4j import GraphDatabase
from neo4j import exceptions
from pyprojroot import here
from dotenv import load_dotenv

In [2]:
# import environment variables
load_dotenv()
URI = os.environ["NEO4J_URI"]
AUTH = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
DATABASE = os.environ["NEO4J_DATABASE"]

In [3]:
def testConnectivity():
    try:
        driver = GraphDatabase.driver(uri=URI, auth=AUTH)
        session = driver.session(database=DATABASE)
        driver.verify_connectivity()
    except exceptions.ServiceUnavailable:
        raise ValueError(
            "URL is incorrect. Could not connect to Neo4j database. Please try again.")
    except exceptions.AuthError:
        raise ValueError(
            "the username and password are incorrect. Could not connect to Neo4j database. Please try again.")
    finally:
        session.close()
        driver.close()

In [4]:
def executeQuery(query: str):
    try:
        driver = GraphDatabase.driver(uri=URI, auth=AUTH)
        session = driver.session(database=DATABASE)
        session.run(query)
        #driver.execute_query(query)
    except exceptions.ServiceUnavailable:
        raise ValueError(
            "URL is incorrect. Could not connect to Neo4j database. Please try again.")
    except exceptions.AuthError:
        raise ValueError(
            "the username and password are incorrect. Could not connect to Neo4j database. Please try again.")
    finally:
        session.close()
        driver.close()

In [ ]:
def init_query()-> str:
    query = """
        LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ahmedsaud-01/neo4J-samples/refs/heads/main/connect-explore-graph-db-python/data/laptops.csv'
        AS row
        MERGE (l:Laptop {id:row.index})
        SET l.model = date(row.released),
            l.brand = row.brand,
            l.rating = toFloat(row.rating),
            l.price = toFloat(row.price)

    """
    return query


executeQuery(init_query())

In [5]:
# clean database
EMPTYDBQRY = """
    MATCH (n)
    DETACH DELETE n
"""


executeQuery(EMPTYDBQRY)

In [6]:

executeQuery("""
    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ahmedsaud-01/neo4J-samples/refs/heads/main/connect-explore-graph-db-python/data/imdb-movies-1990-2024-years.csv'    
    as row LIMIT 50000
    CALL (row){        
        MERGE (m:Movie {id:row.tconst})
            SET m.runtimeMinutes = row.runtimeMinutes,
                m.releasedYear = date(row.startYear),                        
                m.title = row.originalTitle,
                m.averageRatings = toFloat(row.averageRating),
                m.numVotes = toFloat(row.numVotes)
        
        MERGE (p:Person {name:trim(row.personName)})
            SET p.personId = row.nconst
        MERGE (p)-[:ACTED_IN]->(m)
                    
        FOREACH (genre in split(row.genres, ',') |                  // For each genre in the list of genres from the row (split by '|')
            MERGE (g:Genre {name:trim(genre)})                      // Merge a Genre node with the genre's name from the row, trimming any extra spaces
            MERGE (m)-[:OF_GENRE]->(g))                             // Create an IN_GENRE relationship from the Movie to the Genre
        } IN TRANSACTIONS OF 1000 ROWS
""")